Yapılması Gerekenler

-tüm veri setleri "_id" sütunu eklenerek 1den başlamak üzere tüm satırlar indexlenmelidir.  

-sütunlar isimler(df.columns) "CamelCase" ya da "snake_case" olarak standartlaştırılmalıdır. 

-zaman bilgisi içeren sütunlar gerekirse bir araya getirilerek ISO8601 date formata uygun hale getirilmelidir. Bunun için python datetime kütüphanesinin isodate fonksiyonu kullanılabilir. 

-numeric girdiler ondalık, binlik ve milyonluk basamaklara doğru işaretlerle ayrılmalıdır. (1.000.200,75) ya da (1 000 200,75) ya da (1 000.200,75)

-tüm veri setleri csv formatında çıktı alınmalıdır. 

aktýfle*þ*žtýrýlmý*þ*ž = aktifleştirilmiş

In [1]:
import pandas as pd
import os
import csv
import cchardet
import unicodedata
import datetime
from datetime import datetime

# Çıktı Verme Fonsiyonları

In [2]:
def sep_to_str(sep): # Seperatorleri loglamak için bir fonksiyon (Tab hatasını düzeltmek için yazıldı)
    if(sep == "	"):
        return "Tab"
    elif (len(sep) == 1):
        return " " + sep
    return sep

# Dosya ön işlemleri

In [5]:
def get_encoding(filepath): #Dosyanın hangi codecle oluşturulduğunu cchardet kütüphanesi ile beliriyorum
    with open(filepath, "rb") as rawData:
        return cchardet.detect(rawData.read())["encoding"]


In [6]:
def get_seperator(filepath, enc): # csv dosyaları için sniffer kullanarak seperator tespit ediyorum.
    sniffer = csv.Sniffer()
    with open(filepath, "r", encoding=enc) as data:
        return sniffer.sniff(data.read()).delimiter

# Veri kontrol işlemleri

In [7]:
def add_id_column(dataframe, add_if_exist=False): # İd kolonu ekleme fonsiyonu. add_if_exist => id kolonu varsada eklensin mi ?
    if(add_if_exist): # eğer id kolonu varsada eklensin => başka hiçbir koşula bakılmaksızın.
        dataframe.insert(0, '_id', range(1, len(dataframe) + 1))
    else:
        column = dataframe[dataframe.columns[0]]
        proba_col_0,proba_col_1 = list(range(len(column))), list(range(1, len(column)+1)) # olabilecek id kolonları => 0 - uzunluk veya 1 - uzunluk +1
        col_values = list(column)

        if(col_values != proba_col_0 and col_values != proba_col_1): # eğer id kolonu varsa eklenmesin => id kolonu varmı bak.
            dataframe.insert(0, '_id', range(1, len(dataframe)+ 1))
        else:
            dataframe.rename(columns={str(column):"_id"})# try except at

In [8]:
def replace(text):
    normalized = unicodedata.normalize('NFD', text)
    return u"".join([c for c in normalized if not unicodedata.combining(c)]).replace("", "")


def convert_columns_to_snake(dataframe):
    new_columns = []
    for column in dataframe.columns:
        new_columns.append(replace(str(column).lower().strip().replace(" ", "_")))
    dataframe.columns = new_columns

In [9]:
def is_include_header(dataframe, override = False):
    count = ''.join(list(dataframe.columns)).lower().count("unnamed")
    if  len(dataframe.columns)/2 < count:
        if (override):
            dataframe.rename(columns=dataframe.iloc[0], inplace=True)
            dataframe.drop(0, inplace=True)
        return True
    return False

In [10]:
def is_include_datetime(dataframe, override = False):
    is_overrided = False
    resultStr = ""
    for col in dataframe.columns:
        success_converted_rows_count = 0
        if dataframe[col].dtype == 'object':
            for data in dataframe[col]:
                try:
                    pd.to_datetime(data)
                    success_converted_rows_count += 1
                except ValueError:
                    pass
            resultStr += str(col) + "->" + str(success_converted_rows_count) + " || "
            try:
                if (override):
                    dataframe[col] = pd.to_datetime(dataframe[col])
            except ValueError:
                pass
    return resultStr

In [11]:
def is_include_numeric(dataframe, override = False):
    is_overrided = False
    for col in dataframe.columns:
        if dataframe[col].dtype == 'object':
            try:
                tempCol = pd.to_numeric(str(dataframe[col]).replace(',','.'))
                if (override):
                    dataframe[col] = tempCol
                is_overrided = True
            except ValueError:
                pass
    return is_overrided

# Genel Fonksiyonlar

In [12]:
def get_general_info(file, override, get_output=False):
    filepath = "./content/" + file
    enc = get_encoding(filepath)
    sep = get_seperator(filepath, enc)
    dataframe = pd.read_csv(filepath, encoding=enc, sep=sep)

    include_header = is_include_header(dataframe, override)
    include_datetime = is_include_datetime(dataframe, override)
    include_numeric = is_include_numeric(dataframe, override)
    convert_columns_to_snake(dataframe)
    add_id_column(dataframe)

    if(get_output):
        try:
            dataframe.to_csv("./_output/" + file, index=False) #, sep=";"
        except FileNotFoundError:
            os.system("mkdir _output")
            dataframe.to_csv("./_output/" + file, index=False) #, sep=";"
    return {"Name":file, "Number of Rows" : len(dataframe), "is_na": dataframe.isna().sum().sum() , "Encoding":enc, "Sep":sep_to_str(sep), "Header?":include_header, "Datetime?":include_datetime, "Numeric?":include_numeric, "Columns":dict(dataframe.dtypes)}

In [13]:
files = os.listdir("./content/")

result_dataframe = pd.DataFrame(columns=["Name", "Number of Rows", "is_na", "Encoding", "Sep", "Header?", "Datetime?", "Numeric?", "Columns"])

for file in files:
    if(file.endswith(".csv")):
        result_dataframe.loc[result_dataframe.shape[0]] = get_general_info(file, True, True)

result_dataframe

,Name,Number of Rows,is_na,Encoding,Sep,Header?,Datetime?,Numeric?,Columns
0,138_202203_sehirtiyatroları2019.csv,116,0,ISO-8859-9,;,False,TARİH->116 || GÜN->0 || SAAT->0 || OYUN->0 || ...,False,"{'_id': int64, 'tarih': datetime64[ns], 'gun':..."
1,144_202203_abonesayisi.csv,73,146,ISO-8859-9,;,True,Tarih->73 || Değer->0 || Birim->0 ||,False,"{'_id': int64, 'tarih': datetime64[ns], 'deger..."
2,145_202204_tuketilensumiktari (1).csv,73,0,ISO-8859-9,;,True,Tarih->73 || Değer ->0 || Birim->0 ||,False,"{'_id': int64, 'tarih': datetime64[ns], 'deger..."
3,146_202203_atiksuaritma.csv,41,0,ISO-8859-9,;,True,Tarih->41 || Değer->0 || Birim->0 ||,False,"{'_id': int64, 'tarih': datetime64[ns], 'deger..."
4,148_202203_ciftciegitimenvanteri.csv,32,18,ISO-8859-9,;,False,İLÇESİ->1 ||,False,"{'_id': int64, 's.no': float64, 'ilcesi': obje..."
5,149_202203_devreyegirengeslistesi.csv,809,0,ISO-8859-9,;,False,İl->0 || İlçe->0 || Mahalle/Köy->0 || Kaynak T...,False,"{'_id': int64, 'il': object, 'ilce': object, '..."
6,150_202203_internetkullanim.csv,14,28,ISO-8859-13,;,False,Tarih->0 ||,False,"{'_id': int64, 'tarih': object, 'android': int..."
7,151_202203_bisikletamirbakımistasyonukoordinat...,27,36,ISO-8859-9,;,False,Name->3 ||,False,"{'_id': int64, 'name': object, 'enlem': float6..."
8,152_202203_paylaşımlıbisikletistasyonlarıkonum...,40,0,ISO-8859-9,;,False,Name->0 || Bölge->0 ||,False,"{'id': int64, 'x': float64, 'y': float64, 'nam..."


In [14]:
result_dataframe.to_csv('result.csv') #, sep=";"

In [15]:
files = os.listdir("./_output/")
for file in files:
    if(file.endswith(".csv")):
        dataframe = pd.read_csv("./_output/" + file)
        try:
            dataframe.to_csv("./_acikverikapsul/" + file[:-4] + "(k)" + ".csv", index=False) #, sep=";"
        except FileNotFoundError:
            os.system("mkdir _acikverikapsul")
            dataframe.to_csv("./_acikverikapsul/" + file[:-4] + "(k)" + ".csv", index=False) #, sep=";"